In [1]:
from tensorflow.keras import layers, optimizers, callbacks, utils, losses, metrics, backend as K
from sklearn import metrics as skmetrics, preprocessing
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
import os, gc, joblib, warnings
import tensorflow_addons as tfa
import tensorflow as tf
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 20))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.25)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [3]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')


In [4]:
test_id = test.id.values
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)

In [5]:
sparse_features = [f for f in train.columns if 'cat' in f]
dense_features = [feat for feat in train.columns if  feat not in sparse_features+['target']]

for col in sparse_features:
    train_only = list(set(train[col].unique()) - set(test[col].unique()))
    test_only = list(set(test[col].unique()) - set(train[col].unique()))
    both = list(set(test[col].unique()).union(set(train[col].unique())))
    train.loc[train[col].isin(train_only), col] = np.nan
    test.loc[test[col].isin(test_only), col] = np.nan
    mode = train[col].mode().values[0]
    train[col] = train[col].fillna(mode)
    test[col] = test[col].fillna(mode)
    

In [6]:
for feat in dense_features:
    test[feat] = np.clip(test[feat], train[feat].min(), train[feat].max())

In [7]:
test["target"] = -1
data = pd.concat([train, test]).reset_index(drop=True)

for c in dense_features:
    data[f'q_{c}'], bins_ = pd.qcut(data[c], 25, retbins=True, labels=[i for i in range(25)])
    data[f'q_{c}'] = data[f'q_{c}'].astype('str')
    sparse_features.append(f'q_{c}')

In [8]:
features = sparse_features
for feat in features:
    lbl_enc = preprocessing.OrdinalEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].fillna('-1').values.reshape(-1,1).astype(str))
    
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

In [9]:
oof_preds = np.zeros((len(train)))
bagged_oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))
bagged_test_preds = np.zeros((len(test)))
learning_rate = 1e-3
label_smoothing = 0.0
Verbose = 0
n_splits = [10, 15]
n_bags = 2
seeds = [2021, 2021]

In [10]:
for bag in range(n_bags):
    print(f'Iteration {bag+1} splits {n_splits[bag]} seed {seeds[bag]}')
    for fold, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=n_splits[bag], shuffle=True, random_state=seeds[bag]).split(train, train.target.values)):
        X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train, y_test = X_train.target.values, X_test.target.values
        model = create_model(data, features)
        model.compile(
            optimizer=tfa.optimizers.SWA(tf.keras.optimizers.Adam(learning_rate=learning_rate)),
            loss=losses.BinaryCrossentropy(label_smoothing=label_smoothing),
            metrics=metrics.AUC(name="AUC"),
        )

        X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
        X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
        es = callbacks.EarlyStopping(monitor='val_AUC', min_delta=0.000001, patience=10, verbose=Verbose, mode='max', baseline=None, restore_best_weights=True)
        sb = callbacks.ModelCheckpoint('./nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose, monitor='val_AUC',mode='max')
        plateau  = callbacks.ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=2, verbose=Verbose,
                                        mode='max', min_delta=0.0001, cooldown=0, min_lr=1e-7)
        model.fit(X_train,
                  y_train,
                  validation_data=(X_test, y_test),
                  verbose=Verbose,
                  batch_size=2048,
                  callbacks=[es, sb, plateau],
                  epochs=120
             )
        valid_fold_preds = model.predict(X_test)
        test_fold_preds = model.predict(test_data)
        oof_preds[test_index] = rankdata(valid_fold_preds.ravel())/len(X_test)
        test_preds += rankdata(test_fold_preds.ravel() / n_splits[bag])/len(test)
        print(f'fold {fold+1} AUC : {skmetrics.roc_auc_score(y_test, valid_fold_preds)}')
        K.clear_session()
    print(f'Overall AUC of Iteration {bag+1} = {skmetrics.roc_auc_score(train.target.values, oof_preds)}')
    np.save(f'oof_preds_{bag}',oof_preds)
    np.save(f'test_preds_{bag}',test_preds)
    bagged_test_preds += test_preds / n_bags
    bagged_oof_preds += oof_preds / n_bags

Iteration 1 splits 10 seed 2021
fold 1 AUC : 0.8963407918659074
fold 2 AUC : 0.8947308782910746
fold 3 AUC : 0.892532826377632
fold 4 AUC : 0.8911952262290276
fold 5 AUC : 0.8935665194812421
fold 6 AUC : 0.8947203470848193
fold 7 AUC : 0.8959554484381071
fold 8 AUC : 0.8945549007217966
fold 9 AUC : 0.8917274244891114
fold 10 AUC : 0.8933755149084286
Overall AUC of Iteration 1 = 0.8938699838056556
Iteration 2 splits 15 seed 2021
fold 1 AUC : 0.895676609440068
fold 2 AUC : 0.8965566290071578
fold 3 AUC : 0.8942731748426278
fold 4 AUC : 0.8911695546395435
fold 5 AUC : 0.8956053219166467
fold 6 AUC : 0.8903794919148688
fold 7 AUC : 0.8930856186910237
fold 8 AUC : 0.8947709612892698
fold 9 AUC : 0.8966262601742172
fold 10 AUC : 0.8953878818345529
fold 11 AUC : 0.8987638988108487
fold 12 AUC : 0.8924355584423174
fold 13 AUC : 0.8936255517247417
fold 14 AUC : 0.8906065840868588
fold 15 AUC : 0.8930093716881689
Overall AUC of Iteration 2 = 0.8941314892986865


In [11]:
print("Overall AUC={}".format(skmetrics.roc_auc_score(train.target.values, bagged_oof_preds)))

Overall AUC=0.8948705678546909


In [12]:
print('Saving submission file')
submission = pd.DataFrame.from_dict({
    'id': test_id,
    'target': bagged_test_preds,
})
submission.to_csv('submission.csv', index=False)
submission.head(3)

Saving submission file


,id,target
0,5,8.735075
1,6,13.917735
2,8,1.140623


In [13]:
best_sub = pd.read_csv('../input/tps-mar-2021/submission_lp.csv')
submission['target'] = (rankdata(submission.target) * 0.275 + rankdata(best_sub.target) * 0.725)/len(submission)
submission.to_csv('submission 1.csv', index=False)
submission.head(3)

,id,target
0,5,0.499220
1,6,0.778768
2,8,0.063087


In [14]:
best_sub1 = pd.read_csv('../input/tps-mar-2021/submission_lp.csv')
submission['target'] = (rankdata(submission.target) * 0.2 + rankdata(best_sub1.target) * 0.8)/len(submission)
submission.to_csv('submission 2.csv', index=False)
submission.head(3)

,id,target
0,5,0.498475
1,6,0.773401
2,8,0.062887


In [15]:
best_sub21 = pd.read_csv('../input/tps-mar-2021/submission_lp.csv')
submission['target'] = (rankdata(submission.target) * 0.1 + rankdata(best_sub21.target) * 0.9)/len(submission)
submission.to_csv('submission 31.csv', index=False)
submission.head(3)

,id,target
0,5,0.497966
1,6,0.772076
2,8,0.063636


In [16]:
best_sub2 = pd.read_csv('../input/tps-mar-2021/submission_pc.csv')
submission['target'] = (rankdata(submission.target) * 0.1 + rankdata(best_sub2.target) * 0.9)/len(submission)
submission.to_csv('submission 3.csv', index=False)
submission.head(3)

,id,target
0,5,0.497561
1,6,0.769082
2,8,0.063829


In [17]:
best_sub3 = pd.read_csv('../input/tps-mar-2021/submission_pc.csv')
submission['target'] = (rankdata(submission.target) * 0.2 + rankdata(best_sub3.target) * 0.8)/len(submission)
submission.to_csv('submission 4.csv', index=False)
submission.head(3)

,id,target
0,5,0.497524
1,6,0.768819
2,8,0.063849


In [18]:
best_sub4 = pd.read_csv('../input/tps-mar-2021/submission_pc.csv')
submission['target'] = (rankdata(submission.target) * 0.275 + rankdata(best_sub4.target) * 0.725)/len(submission)
submission.to_csv('submission 5.csv', index=False)
submission.head(3)

,id,target
0,5,0.497518
1,6,0.768771
2,8,0.063854


Baseline :
* https://www.kaggle.com/siavrez/kerasembeddings/data#data